In [1]:
pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.3/450.3 kB 6.6 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
#importing packages
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests                                      # reading data
from io import StringIO

from sklearn.datasets import fetch_openml            # common data set access
from sklearn.preprocessing import StandardScaler     # scaling transform
from sklearn.model_selection import train_test_split # validation tools
from sklearn.metrics import zero_one_loss as J01

import sklearn.tree as tree

# Fix the random seed for reproducibility
# !! Important !! : do not change this
seed = 1234
np.random.seed(seed)  

In [17]:
# Reading in the CSV file
dia_data = pd.read_csv("data/diabetic_data.csv")

#print(dia_X.shape)
# View the first 5 rows
print(dia_data.head())
#print(f"NumFeatures: {len(dia_data.columns)} \nFeatures: {dia_data.columns}")
#print(f"Summary: {dia_data.info}")

   encounter_id  patient_nbr             race  gender      age weight  \
0       2278392      8222157        Caucasian  Female   [0-10)      ?   
1        149190     55629189        Caucasian  Female  [10-20)      ?   
2         64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3        500364     82442376        Caucasian    Male  [30-40)      ?   
4         16680     42519267        Caucasian    Male  [40-50)      ?   

   admission_type_id  discharge_disposition_id  admission_source_id  \
0                  6                        25                    1   
1                  1                         1                    7   
2                  1                         1                    7   
3                  1                         1                    7   
4                  1                         1                    7   

   time_in_hospital  ... citoglipton insulin  glyburide-metformin  \
0                 1  ...          No      No                   No

In [18]:
#data = dia_data.to_numpy()
X, Y = dia_data.iloc[:,:-1], dia_data.iloc[:,-1]
#train 70% temp 30%
Xtr, Xtemp, Ytr, Ytemp = train_test_split(X, Y, test_size=0.3, random_state=seed)


# validation: 10% Test: 20%
Xva, Xtest, Yva, Ytest = train_test_split(Xtemp, Ytemp, test_size=0.67, random_state=seed)

print(Xtr.shape, Ytr.shape)
print(Xva.shape, Yva.shape)
print(Xtest.shape, Ytest.shape)

(71236, 49) (71236,)
(10074, 49) (10074,)
(20456, 49) (20456,)


In [20]:
Xtr_Abhinand=Xtr.iloc[:,1:17]

In [21]:
Xtr_Abhinand

,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency
47866,25174899,AfricanAmerican,Female,[10-20),?,1,1,7,3,MD,InternalMedicine,37,1,13,0,0
24673,298521,AfricanAmerican,Female,[90-100),?,1,14,7,3,?,?,57,0,10,0,0
61033,77827617,Caucasian,Male,[80-90),[75-100),2,6,1,4,MC,Cardiology,71,0,11,0,0
74444,38516562,?,Male,[80-90),?,1,1,7,1,MC,?,29,0,6,0,0
56971,46248840,AfricanAmerican,Female,[30-40),?,1,1,7,10,MD,?,49,1,18,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89460,50693553,Caucasian,Male,[80-90),?,1,6,7,9,MC,Emergency/Trauma,72,4,22,0,0
60620,66287709,Caucasian,Male,[50-60),?,5,1,1,3,HM,?,1,3,9,1,0
34086,24745671,Caucasian,Female,[80-90),?,2,1,4,5,MC,Cardiology,14,4,23,0,0
58067,59602239,Caucasian,Female,[80-90),?,1,1,7,3,CM,?,45,6,13,0,0


In [30]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

Xtr_Abhinand = Xtr_Abhinand.replace('?', np.nan)

numerical_columns = Xtr_Abhinand.select_dtypes(include=['number']).columns
categorical_columns = Xtr_Abhinand.select_dtypes(include=['object']).columns

numerical_imputer = SimpleImputer(strategy='median')
Xtr_Abhinand[numerical_columns] = numerical_imputer.fit_transform(Xtr_Abhinand[numerical_columns])


categorical_imputer = SimpleImputer(strategy='most_frequent')
Xtr_Abhinand[categorical_columns] = categorical_imputer.fit_transform(Xtr_Abhinand[categorical_columns])


In [41]:
Xtr_Abhinand=Xtr_Abhinand.reset_index()
Xtr_Abhinand

,index,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency
0,47866,25174899.0,AfricanAmerican,Female,[10-20),[75-100),1.0,1.0,7.0,3.0,MD,InternalMedicine,37.0,1.0,13.0,0.0,0.0
1,24673,298521.0,AfricanAmerican,Female,[90-100),[75-100),1.0,14.0,7.0,3.0,MC,InternalMedicine,57.0,0.0,10.0,0.0,0.0
2,61033,77827617.0,Caucasian,Male,[80-90),[75-100),2.0,6.0,1.0,4.0,MC,Cardiology,71.0,0.0,11.0,0.0,0.0
3,74444,38516562.0,Caucasian,Male,[80-90),[75-100),1.0,1.0,7.0,1.0,MC,InternalMedicine,29.0,0.0,6.0,0.0,0.0
4,56971,46248840.0,AfricanAmerican,Female,[30-40),[75-100),1.0,1.0,7.0,10.0,MD,InternalMedicine,49.0,1.0,18.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,89460,50693553.0,Caucasian,Male,[80-90),[75-100),1.0,6.0,7.0,9.0,MC,Emergency/Trauma,72.0,4.0,22.0,0.0,0.0
71232,60620,66287709.0,Caucasian,Male,[50-60),[75-100),5.0,1.0,1.0,3.0,HM,InternalMedicine,1.0,3.0,9.0,1.0,0.0
71233,34086,24745671.0,Caucasian,Female,[80-90),[75-100),2.0,1.0,4.0,5.0,MC,Cardiology,14.0,4.0,23.0,0.0,0.0
71234,58067,59602239.0,Caucasian,Female,[80-90),[75-100),1.0,1.0,7.0,3.0,CM,InternalMedicine,45.0,6.0,13.0,0.0,0.0


In [32]:
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
Xtr_categorical = Xtr_Abhinand[categorical_columns]
Xtr_encoded = encoder.fit_transform(Xtr_categorical)

`sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [37]:
Xtr_encoded_df

,race_AfricanAmerican,race_Asian,race_Caucasian,race_Hispanic,race_Other,gender_Female,gender_Male,gender_Unknown/Invalid,age_[0-10),age_[10-20),...,medical_specialty_Surgery-General,medical_specialty_Surgery-Maxillofacial,medical_specialty_Surgery-Neuro,medical_specialty_Surgery-Pediatric,medical_specialty_Surgery-Plastic,medical_specialty_Surgery-PlasticwithinHeadandNeck,medical_specialty_Surgery-Thoracic,medical_specialty_Surgery-Vascular,medical_specialty_SurgicalSpecialty,medical_specialty_Urology
0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71232,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71233,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71234,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
concatDF=pd.concat([Xtr_Abhinand, Xtr_encoded_df],axis=1)

In [44]:
concatDF=concatDF.drop(categorical_columns, axis=1)

In [45]:
concatDF

,index,patient_nbr,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,...,medical_specialty_Surgery-General,medical_specialty_Surgery-Maxillofacial,medical_specialty_Surgery-Neuro,medical_specialty_Surgery-Pediatric,medical_specialty_Surgery-Plastic,medical_specialty_Surgery-PlasticwithinHeadandNeck,medical_specialty_Surgery-Thoracic,medical_specialty_Surgery-Vascular,medical_specialty_SurgicalSpecialty,medical_specialty_Urology
0,47866,25174899.0,1.0,1.0,7.0,3.0,37.0,1.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,24673,298521.0,1.0,14.0,7.0,3.0,57.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,61033,77827617.0,2.0,6.0,1.0,4.0,71.0,0.0,11.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,74444,38516562.0,1.0,1.0,7.0,1.0,29.0,0.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,56971,46248840.0,1.0,1.0,7.0,10.0,49.0,1.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71231,89460,50693553.0,1.0,6.0,7.0,9.0,72.0,4.0,22.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71232,60620,66287709.0,5.0,1.0,1.0,3.0,1.0,3.0,9.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71233,34086,24745671.0,2.0,1.0,4.0,5.0,14.0,4.0,23.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71234,58067,59602239.0,1.0,1.0,7.0,3.0,45.0,6.0,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
concatDF.to_csv('/users/abhinandganesh/Desktop/Diabetes_ENCODED_1-16.csv')